We should be able to extract options from the field list

In [5]:
import pytest
import json
import ipywidgets as widgets
from enum import Enum
from data_helpers.cls_construction import *
from data_helpers.cls_parsing import dict_to_cls
from data_helpers.encoders import AdvancedJsonDecoder, AdvancedJsonEncoder


In [6]:

# example_fields = [
#     Field('main', 'foo', 'Foo', str, 'Example field description a', True, None, []),
#     NumberField('main', 'bar', 'Bar', int, 'Example number field', False, 9, min=3, max=33, step=3),
#     Select('main', 'sel', 'Sel', str, 'Example field', False, None,
#            options=[Option('a', 'A', []), Option('b', 'B', [])]),
#     Field('other', 'bar', 'Bar', str, 'Example field', True, "default_bar", []),
# ]

# example_groups = set(f.group for f in example_fields)

example_fields = [
    Field('foo', 'Foo', str, 'Example field description a', True),
    Select(
        'sel', 'Sel', str, 'Example field', True,
        default='A',
        options=[
               Option('a', 'A', []),
               Option('b', 'B', [])
        ]
    ),

    NumberField('bar', 'Bar', int, 'Example number field', True, 9, min=3, max=33, step=3),
    ListField(Field('foos', 'Foos', int, 'Example list', default=[])),
    Group('main', 'Main', True, [
        Field('inner', 'Inner', str),
    ]),
    Group('other', 'Other', True, [
        Field('inner_b', 'InnerB', str),
    ]),
]

example_group = Group('GeneratedType', 'Generated Type', True, example_fields)


In [7]:
# Simply display each field as grouped text input
from ipywidgets import Box, HBox, VBox, Layout
field_widgets = {}
input_layout = Layout( width='40%')
label_layout = Layout( width='30%', display="flex", justify_content="flex-end")
desc_layout = Layout( width='30%', display="flex", justify_content="flex-start")
field_container_layout = Layout(display='flex', width='100%', align_items='stretch', flex_flow='horiz')
for field in example_fields:
    if field.group not in field_widgets:
        field_widgets[field.group] = []
    label = widgets.Label(field.label, layout=label_layout)
    input_field = widgets.Text(layout=input_layout)
    desc = widgets.Label(field.desc, layout=desc_layout)
    contained = Box([label, input_field, desc], layout=field_container_layout)
    field_widgets[field.group].append(contained)

container_layout = Layout(
    display='flex',
    border='solid 0.1px',
    flex_flow='column',
    align_items='stretch',
)
boxes = VBox([Box(children=[widgets.Label(k), *v], layout=container_layout) for k,v in field_widgets.items()], layout=container_layout)
boxes
# field_widgets


AttributeError: 'Field' object has no attribute 'group'

In [ ]:
# Display sections based on initial inputs
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

label = widgets.Label("hello")
section = Box([
    widgets.Label("A"),
    widgets.Label("B"),
    widgets.Label("C"),
    widgets.Label("D"),
])
def on_button_clicked(b):
    section.layout.visibility = 'visible' if section.layout.visibility == 'hidden' else 'hidden'
    
button.on_click(on_button_clicked)
display(Box([label, button, section]))
display(output)

Box(children=(Label(value='hello'), Button(description='Click Me!', style=ButtonStyle()), Box(children=(Label(…

Output()

In [ ]:
# Linked widgets
from ipywidgets import widgets  

# Create text widget for output
output_text = widgets.Text()

# Create text widget for input
input_text = widgets.Text()

# Define function to bind value of the input to the output variable 
def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
# input_text.on_value_change(bind_input_to_output)
input_text.observe(bind_input_to_output, names="value")
# Display input text box widget for input
Box([input_text,output_text])


Box(children=(Text(value=''), Text(value='')))

In [ ]:
output_text

Text(value='')

# Collapseable sections

In [ ]:
# Simply display each field as grouped text input
from ipywidgets import Box, HBox, VBox, Layout
field_widgets = {}
output=widgets.Output()
input_layout = Layout( width='40%')
label_layout = Layout( width='30%', display="flex", justify_content="flex-end")
desc_layout = Layout( width='30%', display="flex", justify_content="flex-start")
field_container_layout = Layout(display='flex', width='100%', align_items='stretch', flex_flow='horiz')
def container_layout(expanded=True): return Layout(
    display='flex',
    border='solid 0.1px',
    flex_flow='column',
    align_items='stretch',
    height='auto',
    visibility='visible',
) if expanded else Layout(
    display='flex',
    border='solid 0.1px',
    flex_flow='column',
    align_items='stretch',
    height='0',
    visibility='hidden',
)

for field in example_fields:
    if field.group not in field_widgets:
        field_widgets[field.group] = []
    label = widgets.Label(field.label, layout=label_layout)
    input_field = widgets.Text(layout=input_layout)
    desc = widgets.Label(field.desc, layout=desc_layout)
    contained = Box([label, input_field, desc], layout=field_container_layout)
    field_widgets[field.group].append(contained)
    # display(f)


sections = []
for k, v in field_widgets.items():
    content = Box(v,layout=container_layout(False))
    header_btn = widgets.Button(description=k)
    sections.append(Box([header_btn, content], layout=container_layout()))
    def on_button_clicked(target, k):
        def inner(b):
            is_visable = target.layout.visibility != 'hidden'
            target.layout = container_layout(False) if is_visable else container_layout(True)
        return inner
    header_btn.on_click(on_button_clicked(content, k))

boxes = VBox(sections, layout=container_layout())
display(output)
display(boxes)
# field_widgets


Output()

# Field Type Specific Fields

In [ ]:
# Simply display each field as grouped text input
input_layout = Layout( width='40%')
label_layout = Layout( width='30%', display="flex", justify_content="flex-end")
desc_layout = Layout( width='30%', display="flex", justify_content="flex-start")
field_container_layout = Layout(display='flex', width='100%', align_items='stretch', flex_flow='horiz')

def get_field(field):
    if isinstance(field, Select):
        return widgets.RadioButtons(
            options=[(opt.label, opt.uid) for opt in field.options],
            layout=input_layout,
            disabled=False
        )
    if isinstance(field, NumberField):
        if field.cls == int:
            return widgets.IntSlider(layout=input_layout, min=field.min, max=field.max, step=field.step)   

    if field.cls == str:
            return widgets.Text(layout=input_layout)
    return widgets.Text(layout=input_layout)

field_widgets = {}
field_inputs = {}

for i, field in enumerate(example_fields):
    if field.group not in field_widgets:
        field_widgets[field.group] = []
        field_inputs[field.group] = []
    label = widgets.Label(field.label, layout=label_layout)
    input_field = get_field(field)
    desc = widgets.Label(field.desc, layout=desc_layout)
    contained = Box([label, input_field, desc], layout=field_container_layout)
    field_widgets[field.group].append(contained)
    field_inputs[field.group].append((i, input_field))

container_layout = Layout(
    display='flex',
    border='solid 0.1px',
    flex_flow='column',
    align_items='stretch',
)
boxes = VBox([Box(children=[widgets.Label(k), *v], layout=container_layout) for k,v in field_widgets.items()], layout=container_layout)
boxes
# field_widgets


# Convert inputs to config file

In [ ]:
for group in field_inputs:
    group_widgets = field_inputs[group]
    for (i, w) in group_widgets:
        print(f"{example_fields[i].label}: {w.value}")

Foo: 
Bar: 3
Sel: a
Bar: 


# Embed graphs in widgets

In [ ]:
from matplotlib import pyplot as plt
# NOTE: Needs ipyml https://matplotlib.org/stable/tutorials/introductory/usage.html?highlight=ipympl#ipympl
output_a = widgets.Output()
with output_a:
  plt.plot([1,2,3])
  plt.show() 
# output_b = widgets.Output()
# with output_b:
#   plt.plot([4,2,3])
# #   plt.show() 
b = Box([
    widgets.Label("hello"),
    widgets.Text(),
    output_a,
    widgets.Label("World"),
    # output_b,
])
display(b)
# display(output_b)


Box(children=(Label(value='hello'), Text(value=''), Output(), Label(value='World')))

# Using Widget Generator Class

In [ ]:
configGenerator = ConfigGeneratorUI(example_fields)
ui = configGenerator.generate_widgets()
display(ui)


# Dynamic field counts
We need to be able to set the number of sub fields based on a slider

In [9]:
inputs = [widgets.Text() for _ in range(10)]
field_count = widgets.IntSlider(min=0, max=len(inputs)-1)
output = widgets.Output()
output_text = widgets.Text()
box_layout = widgets.Layout(
    display="flex",
    flex_flow="column",
    height="auto",
)
input_container = Box(inputs, layout=box_layout)

def limit_inputs_to_slider(sender):
    output_text.value = str(field_count.value)
    for i in inputs:
        i.layout.visibility = "hidden"
        i.layout.max_height = "0px"
    for i in range(field_count.value):
        inputs[i].layout.visibility = "visible"
        i.layout.max_height = "100px"

field_count.observe(limit_inputs_to_slider, names="value")

display(field_count)
display(input_container)
display(output)
display(output_text)

IntSlider(value=0, max=9)

Box(children=(Text(value=''), Text(value=''), Text(value=''), Text(value=''), Text(value=''), Text(value=''), …

Output()

Text(value='')

In [ ]:
inputs = [widgets.Text() for _ in range(10)]
field_count = widgets.IntSlider(min=0, max=len(inputs)-1)
output = widgets.Output()
output_text = widgets.Text()
box_layout = widgets.Layout(
    display="flex",
    flex_flow="column",
    height="auto",
)
input_container = Box(inputs, layout=box_layout)
button = widgets.Button(description="Click Me!")

def limit_inputs_to_slider(sender):
    output_text.value = str(field_count.value)
    for i, inputItem in enumerate(inputs):
        if i > field_count.value:
            inputItem.layout.visibility = "hidden"
            inputItem.layout.max_height = "0"
        else:
            inputItem.layout.visibility = "visible"
            inputItem.layout.max_height = "100"

def on_button_clicked(b):
    for inputItem in inputs:
        inputItem.layout.visibility = "visible"
        inputItem.layout.max_height = "100"
    
button.on_click(on_button_clicked)

field_count.observe(limit_inputs_to_slider, names="value")

display(field_count)
display(input_container)
display(output)
display(output_text)
display(button)


IntSlider(value=0, max=9)

Box(children=(Text(value=''), Text(value=''), Text(value=''), Text(value=''), Text(value=''), Text(value=''), …

Output()

Text(value='')

Button(description='Click Me!', style=ButtonStyle())

In [ ]:
# Display sections based on initial inputs
from IPython.display import display
button = widgets.Button(description="Click Me!")
output = widgets.Output()

label = widgets.Label("hello")
inputs = [
    widgets.Label("A"),
    widgets.Label("B"),
    widgets.Label("C"),
    widgets.Label("D"),
]
section = VBox(inputs)
def on_button_clicked(b):
    inputs[2].layout.visibility = 'visible' if inputs[2].layout.visibility == 'hidden' else 'hidden'
    inputs[2].layout.height = '0' if inputs[2].layout.visibility == 'hidden' else 'auto'
    
button.on_click(on_button_clicked)
display(Box([label, button, section]))
display(output)


Box(children=(Label(value='hello'), Button(description='Click Me!', style=ButtonStyle()), VBox(children=(Label…

Output()

In [12]:
inputs = [widgets.Text() for _ in range(10)]
field_count = widgets.IntSlider(min=0, max=len(inputs)-1)
input_container = widgets.VBox(inputs)

def limit_inputs_to_slider(sender):
    for i, inputItem in enumerate(inputs):
        if i > field_count.value:
            inputItem.layout.visibility = "hidden"
            inputItem.layout.max_height = "0px"
        else:
            inputItem.layout.visibility = "visible"
            inputItem.layout.max_height = "1000px"

field_count.observe(limit_inputs_to_slider, names="value")

display(field_count)
display(input_container)


IntSlider(value=0, max=9)